# Processing with ASteCA

This notebook allows you to process the data for each cluster listed in the UCC using the [ASteCA](https://asteca.github.io/) package. The documentation of AsteCA has more details on how this tool can be used.

The first step is to install the required ASteCA and [pyABC](https://pyabc.readthedocs.io/en/latest/) packages and import them along with other required packages.

In [ ]:
# Install latest published versions of ASteCA and PyABC
!pip install asteca
!pip install pyabc

# Import required packages
import pandas as pd
from urllib.error import HTTPError
import numpy as np
import matplotlib.pyplot as plt
import os
import tempfile
import datetime as dt
import asteca
import pyabc

First we **define the name of the cluster to be analyzed**. In this example we select the cluster `Blanco 1` (it doesn't matter if you use upper or lower case):

In [ ]:
cluster = "blanco 1"

The next step is to load the cluster data into a `pandas` dataframe. To do this we first need to define the function that handles the loading.


In [ ]:
def load_cluster(cluster, db_id):
    """Function that handles loading a cluster's datafile in any of the
    UCC, HUNT23 or CANTAT20 databases, from the UCC repositories."""
    # Proper file name format
    file_name = cluster.lower().replace('_', '').replace(' ', '').replace('-', '')
    # Proper databse name format
    _db_id = '_'+db_id if db_id != 'UCC' else ''
    # Check all repositories where datafiles are stored
    for qfolder in ('1N', '1P', '2N', '2P', '3N', '3P', '4N', '4P'):
        try:
            repo_root = f"https://github.com/ucc23/Q{qfolder}/raw/main/datafiles/"
            path = repo_root + file_name + f"{_db_id}.parquet"
            df = pd.read_parquet(path)
            print(f"Cluster '{cluster}' loaded from '{db_id}' database")
            return df
        except HTTPError:
            pass
    raise ValueError(f"Cluster '{cluster}' not found in {db_id} database")

We can now load a cluster with data from the UCC, [Hunt & Reffert (2023)](https://ui.adsabs.harvard.edu/abs/2023A%26A...673A.114H) (HUNT23) or [Cantat-Gaudin et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A...1C) (CANTAT20), assuming that cluster was processed in either of those articles. To do this select either `UCC`, `HUNT23` or `CANTAT20` in the `db_id` variable below and load.

In [ ]:
db_id = 'UCC'
# db_id = 'HUNT23'
# db_id = 'CANTAT20'

# Load the dataframe
df = load_cluster(cluster, db_id)

The most probable members in the UCC database are stored using a probability cut of `P>0.5`. A minimum number of member stars is set to `25`, so that if less than `25` stars have `P>0.5` then the most probable members are those `25` stars with the largest probability values below 0.5. Other databases handle this differently.

You will need to upload at least a single isochrone file to a new folder named `isochs/`. Once that is done, run the code below to generate the necessary ASteCA objects:

In [ ]:
# Generate a 'cluster' object loading the cluster's data
my_cluster = asteca.cluster(
    obs_df=df,
    ra='RA_ICRS',
    dec='DE_ICRS',
    magnitude="Gmag",
    e_mag="e_Gmag",
    color="BP-RP",
    e_color='e_BP-RP'
)

# Load the isochrone(s) file(s). The file(s) must be located in a 'isochs/' folder within this Colab session
isochs = asteca.isochrones(
    model="PARSEC",
    isochs_path="isochs/",
    magnitude="Gmag",
    color=("G_BPmag", "G_RPmag"),
    magnitude_effl=6390.7,
    color_effl=(5182.58, 7825.08),
)

# Instantiate a 'synthetic' object using the isochrones
synthcl = asteca.synthetic(isochs)

# Define the paramters that are fixed, not fitted
fix_params = {"alpha": 0.09, "beta": 0.94, "Rv": 3.1, "DR": 0.0, "met": 0.152}

# Calibrate the 'synthetic' object
synthcl.calibrate(my_cluster, fix_params)

Now we set up the priors and functions required to use the pyABC package. This is set to run for 3 minutes but we can of course change this parameter as desired.

In [ ]:
# Set parameters ranges for the priors
dm_min, dm_max = 0, 4
Av_min, Av_max = 0, 1
loga_min, loga_max = 6, 10
# Priors dictionary
priors = {
    "loga": [loga_min, loga_max],
    "dm": [dm_min, dm_max],
    "Av": [Av_min, Av_max],
}
# Load priors into pyABC using uniform distributions for all
priors = pyabc.Distribution(
    **{
        p: pyabc.RV("uniform", priors[p][0], priors[p][1] - priors[p][0])
        for p in priors
    }
)

# Define likelihood
likelihood = asteca.likelihood(my_cluster)
max_lkl = likelihood.max_lkl

def model(fit_params):
    """Generate synthetic cluster."""
    synth_clust = synthcl.generate(fit_params)
    # pyABC expects a dictionary from this function
    return {"data": synth_clust}

def distance(synth, obs):
    """
    The likelihood is maximized for better fitted models but this distance requires
    minimization. Hence we normalize and invert.
    """
    lkl = likelihood.get(synth["data"])
    return 1 - lkl / max_lkl

# Define pyABC parameters
pop_size = 100
max_mins = 3
n_procs = 2
abc = pyabc.ABCSMC(
    model,
    priors,
    distance,
    population_size=pop_size,
    sampler=pyabc.sampler.MulticoreEvalParallelSampler(n_procs=n_procs),
)
# This is a temporary file required by pyABC
db_path = "sqlite:///" + os.path.join(tempfile.gettempdir(), "pyABC.db")
abc.new(db_path)

and finally we perform a pyABC run:

In [ ]:
# Run pyABC
history = abc.run(
    max_walltime=dt.timedelta(hours=0, minutes=max_mins),
)
# Extract last iteration and weights
df, w = history.get_distribution()

# Print effective sample size (ESS) and final minimum distance obtained
print(f"\nESS: {pyabc.weighted_statistics.effective_sample_size(w):.3f}")
final_dist = pyabc.inference_util.eps_from_hist(history)
print(f"Dist: {final_dist:.3f}")

# Define a dictionary of model parameters (those that were not fitted) and their uncertainties
fit_model, fit_model_std = {}, {}
for k in df.keys():
    _median = pyabc.weighted_statistics.weighted_median(df[k].values, w)
    _std = pyabc.weighted_statistics.weighted_std(df[k].values, w)
    fit_model[k] = _median
    fit_model_std[k] = _std
    print(f"{k}: {_median:.4f} +/- {_std:.4f}")

We can use the `asteca.plot` method to generate a simple CMD with the resulting best fir synthetic cluster (and its isochrone) found:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
asteca.plot.cluster(my_cluster, ax1)
isoch_arr = asteca.plot.get_isochrone(synthcl, fit_model)
asteca.plot.synthetic(synthcl, ax2, fit_model, isoch_arr)
plt.show()